</div>
<div align="center">
  <img src="img/evolnode.png" width="800" alt="Fourier reconstruction convergence">
  <p><em> Evolution Node free you from coding and debugging, let LLM evolve your code for you.</em></p>
</div>
<div align="left">
<p><em>Bored of manual coding, a function? EvolNode let LLM automate function design and guide the evolution of it with genetic algorithm. A node here takes a task, input, and output, it uses either code or another LLM to complete the task, ensuring aligned input and output value types and ,.names.Fitness evaluation is done by running the function with a few specified test cases, the more diverse the test case, the better the evolution.
</em></p>
</div>

In [4]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.eoh_evolution import EvolNode
from methods.llm import get_groq_response

# Code + Compilor Task
mp = MetaPrompt("Search for age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.CODE)
test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

test_inputs = [c[0] for c in test_cases]
node = EvolNode(mp, None, None, get_response=get_groq_response, test_cases=test_cases)


# reasoning, code = node.evolve("i1", replace=True)
# input_dict = {"n": 10}
# output_dict = node(input_dict) # Ok we need a output dictionary here as well ...


# Prompt + LLM Task
# mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 
# node = EvolNode(mp, None, None, get_response=get_groq_response)
# node.evolve("i1", replace=True)

# input_dict = {"name": "Dilireba"}
# output_dict = node(input_dict) 

In [2]:
node.evolve("i1", replace=True, num_runs=1) # Wrong error message, STH funny in here

Evaluating fitness: 100%|██████████| 2/2 [00:00<00:00, 12.22it/s]


 - Attempt 1 failed. Fitness: 0.00. Error: --- Compiled 0 out of 2 test cases
--- Passed 0 out of 2 test cases
Input: {'query': 'Dilireba'}, Output is missing or of wrong type, Expected: {'results': ['stuff about Dilireba']}
Input: {'query': 'ChengXiao'}, Output is missing or of wrong type, Expected: {'results': ['stuff about ChengXiao']}

Error Message:

**********************************************************************
  Resource punkt not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt/PY3/english.pickle

  Searched in:
    - '/Users/fangyuanyu/nltk_data'
    - '/opt/homebrew/anaconda3/nltk_data'
    - '/opt/homebrew/anaconda3/share/nltk_data'
    - '/opt/homebrew/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '

Evaluating fitness: 100%|██████████| 2/2 [00:00<00:00, 14.47it/s]


 - Attempt 2 failed. Fitness: 0.00. Error: --- Compiled 0 out of 2 test cases
--- Passed 0 out of 2 test cases
Input: {'query': 'Dilireba'}, Output is missing or of wrong type, Expected: {'results': ['stuff about Dilireba']}
Input: {'query': 'ChengXiao'}, Output is missing or of wrong type, Expected: {'results': ['stuff about ChengXiao']}

Error Message:
No module named 'azure'No module named 'azure'
Parse Response Failed...
 - Attempt 3 failed. Fitness: 0.00. Error: 


Evaluating fitness: 100%|██████████| 2/2 [00:00<00:00, 43.02it/s]


 - Attempt 4 failed. Fitness: 0.00. Error: --- Compiled 0 out of 2 test cases
--- Passed 0 out of 2 test cases
Input: {'query': 'Dilireba'}, Output is missing or of wrong type, Expected: {'results': ['stuff about Dilireba']}
Input: {'query': 'ChengXiao'}, Output is missing or of wrong type, Expected: {'results': ['stuff about ChengXiao']}

Error Message:
No module named 'language_model'No module named 'language_model'


Evaluating fitness: 100%|██████████| 2/2 [00:05<00:00,  2.98s/it]

 - Attempt 5 failed. Fitness: 0.00. Error: --- Compiled 2 out of 2 test cases
--- Passed 0 out of 2 test cases
Input: {'query': 'Dilireba'}, Wrong Prediction: {'results': []}, Expected: {'results': ['stuff about Dilireba']}
Input: {'query': 'ChengXiao'}, Wrong Prediction: {'results': []}, Expected: {'results': ['stuff about ChengXiao']}

Error Message:

Evolution failed after 5 attempts.


(None, None)

In [8]:
# Query Engine for relevant nodes 

from methods.eoh_evolution import QueryEngine
qe = QueryEngine()
nodes = qe.query_node("search_google", top_k=5)
relevant_node_str = ("\n\n").join([str(node) for node in nodes])


#### Direct Loading Node

In [1]:
from methods.eoh_evolution import EvolNode 

# node = EvolNode.load("get_celeb_age")
node = EvolNode.load("search_google")
input_dict = {"query": "Dilireba"}
node(input_dict)

/opt/homebrew/anaconda3/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Could not load vllm class, check CUDA support and GPU RAM size


{'result': ('{\n  "Search Result": [\n    {\n      "title": "Dilraba Dilmurat - Wikipedia",\n      "link": "https://en.wikipedia.org/wiki/Dilraba_Dilmurat",\n      "snippet": "Dilraba Dilmurat (Uyghur: \\u062f\\u0649\\u0644\\u0631\\u06d5\\u0628\\u0627 \\u062f\\u0649\\u0644\\u0645\\u06c7\\u0631\\u0627\\u062a, Chinese: \\u8fea\\u4e3d\\u70ed\\u5df4\\u00b7\\u8fea\\u529b\\u6728\\u62c9\\u63d0; born June 3, 1992) is a Chinese actress, singer and model. She is an ethnic Uyghur ...",\n      "position": 1\n    },\n    {\n      "title": "Dilraba Dilmurat (\\u8fea\\u4e3d\\u70ed\\u5df4) - MyDramaList",\n      "link": "https://mydramalist.com/people/7830-dilmurat-dilraba",\n      "snippet": "Dilraba Dilmurat, born in Xinjiang, China, is an award-winning actor, dancer and singer of Uyghur descent.",\n      "sitelinks": [\n        {\n          "title": "Love Beyond the Grave",\n          "link": "https://mydramalist.com/750689-bai-ri-ti-deng"\n        },\n        {\n          "title": "Love on the Tur

In [2]:
print(node.code)


import http.client
import json
import os
from typing import Dict, Any

def _search_google(query: str) -> Dict[str, Any]:
    """
    Use Serper API to search Google for information
    
    Args:
        query (str): The search query
    
    Returns:
        Dict[str, Any]: Parsed JSON response from the API
    """
    conn = http.client.HTTPSConnection("google.serper.dev")
    payload = json.dumps({"q": query})
    headers = {
        'X-API-KEY': os.environ["SERPER_API_KEY"],
        'Content-Type': 'application/json'
    }
    
    try:
        conn.request("POST", "/search", payload, headers)
        res = conn.getresponse()
        data = res.read()
        return json.loads(data.decode("utf-8"))
    except Exception as e:
        print(f"Error occurred during API request: {str(e)}")
        return {}
    finally:
        conn.close()
        
def search_google(query: str) -> str: 
    """ 
    Input query, return search result string from Google
    """
    result = _search_goo

#### Alignment-Check for Evaluation

In [2]:
from methods.llm import get_groq_response 

# Evaluation base on alignment check
node.get_response = get_groq_response 
structure_fitness, function_fitness, issue_summary = node._evaluate_fitness(max_tries=1, num_runs=2)

Evaluating fitness: 100%|██████████| 4/4 [00:13<00:00,  3.39s/it]


In [3]:
print(issue_summary)

--- Compiled 0 out of 4 test cases
--- Passed 0 out of 4 test cases
Given Input: {'name': 'Brad Pitt'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Jennifer Lawrence'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Brad Pitt'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Jennifer Lawrence'}. Can't parse output dictionary from LLM's response.



In [5]:
# how to put a node into another nodes? 
# Ideally I would then ask for the node (which likely fails on many of the test-cases) to come-up with a code, where it could call on some 'pseudo-node'
# These pseudo-nodes are used, very similar to a pseudo-code 




</div>
<div align="center">
  <img src="https://github.com/user-attachments/assets/af98faeb-66d6-4278-af86-67d668d1954e" width="1000" alt="Fourier reconstruction convergence">
  <p><em> But how do we know what are the tasks suitable for our goal? Design of tasks topology is the fundation of planning, let's ask LLM for help on this, too! Evolution Graph autuomate planning by imagning topology of tasks which works best for your goal.</em></p>
</div>

</div>
<div align="center">
  <img src="img/Planning-node.png" width="400" alt="Planning Node Task Decomposition">
  <p><em> But how do we know what are the tasks suitable for our goal? Design of tasks topology is the fundation of planning, let's ask LLM for help on this, too! Evolution Graph autuomate planning by imagning topology of tasks which works best for your goal.</em></p>
</div>


In [1]:
from methods.llm import get_claude_response
from methods.diagram import visualize_plan_dict
from methods.meta_prompt import MetaPlan,extract_python_code, extract_json_from_text

# Step 0. Initialize MetaPlanning Prompt
mp = MetaPlan("Get the age of celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"])

# Step 1: Generate Pseudo-Code for SubTask Decomposition
prompt = mp._get_pseudo_code_prompt() # Pseudo-Code Prompt (Non-implemented functional)
response = get_claude_response(prompt) # Use Strong LLM to build up pseudo-code
code = extract_python_code(response) # Extract Python Code from response 

# Step 2: Generate Planning DAG: Multiple Nodes 
graph_prompt = mp._get_plan_graph_prompt(code) 
plan_response = get_claude_response(graph_prompt)
plan_dict = extract_json_from_text(plan_response)

visualize_plan_dict(plan_dict)

# Step 3: Spawn Multiple Sub-Nodes and Evolve them

# Step 4: Slot Sub Node into context for parent node re-write 

Could not load vllm class, check CUDA support and GPU RAM size


success: successfully compiled d2_output/plan_graph.d2 to d2_output/plan_graph.png in 207.111084ms
success: successfully compiled d2_output/plan_graph_dag.d2 to d2_output/plan_graph_dag.png in 207.980416ms


In [1]:
# Ok we could load this thing now ..

from methods.eoh_evolution import PlanNode 
from methods.llm import get_groq_response 

plan_node = PlanNode.load("get_celeb_age", get_response=get_groq_response)

Could not load vllm class, check CUDA support and GPU RAM size


In [5]:
plan_dict = plan_node.plan_dict

In [6]:
prompt_nodes, code_nodes = plan_node._spawn_nodes(plan_dict)

Generating test cases: 5case [00:05,  1.20s/case]             


--- Generated 5 test cases


Generating test cases: 5case [00:09,  1.99s/case]             


--- Generated 5 test cases


Generating test cases: 5case [00:06,  1.20s/case]             


...

--- Generated 3 test cases


Generating test cases: 5case [00:06,  1.28s/case]             


--- Generated 5 test cases


Generating test cases: 5case [00:10,  2.19s/case]             

--- Generated 5 test cases


In [17]:
# Chicken n Egg problem: - how can you evolve a function which you can't evaluate? 
code_nodes["scrape_celeb_data"].test_cases # probably not fair to expect test-cases for this one to be grounded ... 

[({'name': 'Tom Hanks'}, {'data': '67'}),
 ({'name': 'Emma Watson'}, {'data': '33'}),
 ({'name': 'Leonardo da Vinci'}, {'data': 'Deceased'})]

In [18]:
scrape_node = code_nodes["scrape_celeb_data"]

scrape_node.evolve("i1", replace=True)

Evaluating fitness: 100%|██████████| 15/15 [00:36<00:00,  2.43s/it]


 - Attempt 1 failed. Fitness: 0.00. Error: --- Compiled 10 out of 15 test cases
--- Passed 0 out of 15 test cases
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Input: {'name': 'Emma Watson'}, Pred: {'data': 'Celebrity not found.'}, Expected: {'data': '33'}
Input: {'name': 'Leonardo da Vinci'}, Pred: {'data': 'Celebrity not found.'}, Expected: {'data': 'Deceased'}
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Input: {'name': 'Emma Watson'}, Pred: {'data': 'Celebrity not found.'}, Expected: {'data': '33'}
Input: {'name': 'Leonardo da Vinci'}, Pred: {'data': 'Celebrity not found.'}, Expected: {'data': 'Deceased'}
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Input: {'name': 'Emma Watson'}, Pred: {'data': 'Celebrity not found.'}, Expected: {'data': '33'}
Input: {'name': 'Leonardo da Vinci'}, Pred: {'data': 'Celebrity not found.'}, Expected: {'data': 'Deceased'}
Given I

Evaluating fitness: 100%|██████████| 15/15 [00:00<00:00, 44906.90it/s]


 - Attempt 2 failed. Fitness: 0.00. Error: --- Compiled 0 out of 15 test cases
--- Passed 0 out of 15 test cases
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM

Evaluating fitness: 100%|██████████| 15/15 [00:00<00:00, 31583.61it/s]


 - Attempt 3 failed. Fitness: 0.00. Error: --- Compiled 0 out of 15 test cases
--- Passed 0 out of 15 test cases
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM

Evaluating fitness: 100%|██████████| 15/15 [00:00<00:00, 7772.03it/s]


 - Attempt 4 failed. Fitness: 0.00. Error: --- Compiled 0 out of 15 test cases
--- Passed 0 out of 15 test cases
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM

Evaluating fitness:   0%|          | 0/15 [01:21<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Code Node evolution is bad now, parsing seems to be problematic



In [20]:
code = scrape_node.code
print(code)

defined structured data to locate the celebrity based on their name and then retrieving their corresponding age.}

```python
def scrape_celeb_data(name: str) -> str:
    # Predefined mock data to simulate web data
    celeb_data = {
        "Taylor Swift": 33,
        "Leonardo DiCaprio": 48,
        "Beyoncé": 42,
        "Chris Evans": 42,
        "Scarlett Johansson": 38
    }
    
    # Attempt to find the celebrity's age
    age = celeb_data.get(name)

    # If age not found, return a message
    if age is None:
        return f"Age for {name} not found."
    
    # Return the age as a string
    return f"{name} is {age} years old."


In [10]:
# Evolve Sub-Nodes (hope is not too slow here ...)
# 1. Code success -> end | else -> Prompt | pick best -> return node for specific sub-task

# prompt_nodes, code_nodes = plan_node._spawn_nodes(plan_dict)

In [24]:
# visualize_plan_dict(plan_dict)

plan_dict # Ignore the mode for now, let's use both mode and pick the best performing ones ... 

plan_dict["nodes"]

# Basic Idea: 
# - Build node libraries is the better way to describe this 



[{'task': 'Search for celebrity in database',
  'name': 'search_celebrity_database',
  'inputs': ['name'],
  'input_types': ['str'],
  'outputs': ['celeb_info'],
  'output_types': ['dict'],
  'target': 'Retrieve celebrity information',
  'mode': 'CODE'},
 {'task': 'Extract birth date from celebrity info',
  'name': 'extract_birth_date',
  'inputs': ['celeb_info'],
  'input_types': ['dict'],
  'outputs': ['birth_date'],
  'output_types': ['date'],
  'target': "Get celebrity's birth date",
  'mode': 'CODE'},
 {'task': 'Get current date',
  'name': 'get_current_date',
  'inputs': [],
  'input_types': [],
  'outputs': ['current_date'],
  'output_types': ['date'],
  'target': 'Obtain current date for age calculation',
  'mode': 'CODE'},
 {'task': 'Calculate age based on birth date and current date',
  'name': 'calculate_age',
  'inputs': ['birth_date', 'current_date'],
  'input_types': ['date', 'date'],
  'outputs': ['age'],
  'output_types': ['int'],
  'target': "Determine celebrity's curr